## Corpora

In [ ]:
from ipynb.fs.full.koselleck import *

In [ ]:
C=None
def get_corpus():
    global C
    if C is None:
        C=lltk.load(CNAME)
    return C

In [ ]:
C=get_corpus()

In [ ]:
len(C.meta)

## OCR

In [ ]:
from lltk import get_wordlist

In [ ]:
def measure_ocr_accuracy(txt_or_tokens_or_tokend,lang='en'):
    wordlist={w.lower() for w in get_wordlist(lang=lang)}
    if type(txt_or_tokens_or_tokend) in {str}:
        tokend=Counter(tokenize(txt_or_tokens_or_tokend))
    elif type(txt_or_tokens_or_tokend) in {tuple,list}:
        tokens=Counter(txt_or_tokens_or_tokend)
    elif type(txt_or_tokens_or_tokend) in {dict,Counter}:
        tokend=txt_or_tokens_or_tokend
    else: 
        raise Exception("Function `measure_ocr_accuracy(txt_or_tokens_or_tokend)` must take text string or list of tokens or dictionary.")
    numwords=sum(tokend.values())
    numrecog=sum(
        val for tok,val in tokend.items()
        if tok.lower() in wordlist
    )
    return {
        'num_words':numwords,
        'num_recog':numrecog,
        'ocr_accuracy':numrecog/numwords
    }

In [ ]:
def writeiter_jsonl(fnfn,iterr):
    import jsonlines
    with jsonlines.open(fnfn,'w') as writer:
        for i,dx in enumerate(iterr):
            writer.write(dx)

In [ ]:

def do_measure_ocr_accuracy_corpus(obj):
    idx,path_freqs=obj
    od={'id':idx}
    if not os.path.exists(path_freqs): return od
    with open(path_freqs) as f: d=ujson.load(f)
    odx=measure_ocr_accuracy(d)
    return {**od, **odx}
    
def measure_ocr_accuracy_corpus(self,num_proc=1,**y):
    ofn=os.path.join(self.path_data, 'data.ocr_accuracy.jsonl')
    if os.path.exists(ofn): return pd.read_json(ofn,lines=True)
    objs=[(t.id,t.path_freqs) for t in self.texts()]
    iterr=pmap_iter(
        do_measure_ocr_accuracy_corpus,
        objs,
        num_proc=num_proc,
        **y
    )
    return writeiter_jsonl(ofn, iterr)

In [ ]:
measure_ocr_accuracy_corpus(C,num_proc=4)

In [ ]:
stop

In [223]:
# !cat /home/ryan/lltk_data/corpora/bpo/data/data.ocr_accuracy.jsonl

{"id": "1010/Advertisement/2032695", "num_words": 48885, "num_recog": 42828, "ocr_accuracy": 0.8760969622583614}
{"id": "1010/Advertisement/2032690", "num_words": 42862, "num_recog": 36991, "ocr_accuracy": 0.8630255237739722}
{"id": "1010/Unknown/2032695", "num_words": 65408, "num_recog": 63042, "ocr_accuracy": 0.9638270547945206}
{"id": "1010/News/2032690", "num_words": 9280, "num_recog": 8580, "ocr_accuracy": 0.9245689655172413}
{"id": "1010/News/2032695", "num_words": 3771, "num_recog": 3488, "ocr_accuracy": 0.9249535932113497}
{"id": "1010/Review/2032695", "num_words": 2859, "num_recog": 2677, "ocr_accuracy": 0.9363413781042322}
{"id": "1010/Back_Matter/2032690", "num_words": 244, "num_recog": 195, "ocr_accuracy": 0.7991803278688525}
{"id": "1010/Unknown/2032690", "num_words": 110922, "num_recog": 105975, "ocr_accuracy": 0.9554010926597069}
{"id": "1010/Correspondence/2032695", "num_words": 1005, "num_recog": 958, "ocr_accuracy": 0.9532338308457712}
{"id": "1010/General_Information

In [200]:
measure_ocr_accuracy(d)

{'num_words': 91, 'num_recog': 50, 'ocr_accuracy': 0.5494505494505495}

In [ ]:
stop

In [182]:
# C.meta.query('1710<year<1720').Publication_Title.value_counts()
meta=C.meta.query('1600<=year<2100')

In [183]:
s=meta.Publication_Title.value_counts()
s

The Gentleman's Magazine: and historical chronicle, Jan. 1736-Dec. 1833                                                                                                          811
The Athenaeum                                                                                                                                                                    724
Illustrated Sporting and Dramatic News                                                                                                                                           672
Examiner                                                                                                                                                                         595
The Sketch                                                                                                                                                                       584
                                                                                               

In [184]:
dfpub2=meta.groupby('Publication_Title').agg({'year':['min','median','max']})
dfpub2.columns=['_'.join(x) for x in dfpub2.columns.to_flat_index()]
dfpub2['size']=s
dfpub2=dfpub2.sort_values('size',ascending=False)
dfpub2

,year_min,year_median,year_max,size
Publication_Title,,,,
"The Gentleman's Magazine: and historical chronicle, Jan. 1736-Dec. 1833",1736,1784.0,1833,811
The Athenaeum,1830,1876.0,1921,724
Illustrated Sporting and Dramatic News,1642,1905.0,1943,672
Examiner,1808,1845.0,1881,595
The Sketch,1864,1925.0,1997,584
...,...,...,...,...
"General history of discoveries and improvements : in useful arts, particularly in the great branches of commerce, navigation, and plantation, in all parts of the known world",1726,1726.0,1726,1
Fortnightly,1934,1934.0,1934,1
"The History of learning, 1694",1694,1694.0,1694,1


In [186]:
dfpub2.query('year_min<1700').sort_values('year_min')

,year_min,year_median,year_max,size
Publication_Title,,,,
Illustrated Sporting and Dramatic News,1642,1905.0,1943,672
Heraclitus ridens,1681,1703.0,1718,15
"Democritus ridens, or, Comus and Momus",1681,1681.0,1681,3
"The Observator in question and answer, Apr. 1681- July 1681",1681,1681.0,1681,6
The Observator in dialogue,1682,1685.0,1687,20
"Weekly memorials for the ingenious : or, An account of books lately set forth in several languages",1682,1682.0,1683,10
"Weekly lampoon; or, Satyrical reflections on the last weeks publick news letters and observator, Oct. 1690",1690,1690.0,1690,3
"The History of learning, or, An abstract of several books lately published, as well abroad, as at home / By one of the two authors of the Universal and historical bibliotheque",1691,1691.0,1691,2
Athenian gazette,1691,1693.0,1697,36


## Corp info

In [161]:
# C.meta.path_freqs

In [163]:
C.meta[C.meta.Publication_Title.str.contains('Spectator')]

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,ObjectType,ISSN,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus
id,,,,,,,,,,,,,,,,,,,,,
1711/Unknown/18038,,Untitled item,1711,17110303,6319128,18038,The Spectator (1711),London,Sam. Buckley.,,...,,1358-6149,ENG,Historical Periodicals,1244,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Unknown/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Unknown/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Unknown/18038.xml,1711,BPO
1711/Back_Matter/18038,,Back matter,1711,17110303,6319159,18038,The Spectator (1711),London,Sam. Buckley.,,...,Back Matter,1358-6149,ENG,Historical Periodicals,29,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Back_Matter/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Back_Matter/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Back_Matter/18038.xml,1711,BPO
1711/Review/18038,,Book review,1711,17110305,6313768,18038,The Spectator (1711),London,Sam. Buckley.,,...,Review,1358-6149,ENG,Historical Periodicals,1704,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Review/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Review/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Review/18038.xml,1711,BPO
1711/Advertisement/18038,,Advertisement,1711,17110306,6317135,18038,The Spectator (1711),London,Sam. Buckley.,,...,Advertisement,1358-6149,ENG,Historical Periodicals,833,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Advertisement/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Advertisement/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Advertisement/18038.xml,1711,BPO
1711/Correspondence/18038,,Letter,1711,17110309,6399178,18038,The Spectator (1711),London,Sam. Buckley.,,...,Correspondence,1358-6149,ENG,Historical Periodicals,1187,/home/ryan/lltk_data/corpora/bpo/freqs/1711/Correspondence/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/Correspondence/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/Correspondence/18038.xml,1711,BPO
1711/General_Information/18038,,ERRATA.,1711,17110416,6314035,18038,The Spectator (1711),London,Sam. Buckley.,,...,General Information,1358-6149,ENG,Historical Periodicals,32,/home/ryan/lltk_data/corpora/bpo/freqs/1711/General_Information/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1711/General_Information/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1711/General_Information/18038.xml,1711,BPO
1712/Correspondence/18038,,Letter,1712,17120101,6322677,18038,The Spectator (1711),London,Sam. Buckley.,,...,Correspondence,1358-6149,ENG,Historical Periodicals,1652,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Correspondence/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Correspondence/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Correspondence/18038.xml,1712,BPO
1712/Back_Matter/18038,,Back matter,1712,17120101,6413487,18038,The Spectator (1711),London,Sam. Buckley.,,...,Back Matter,1358-6149,ENG,Historical Periodicals,39,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Back_Matter/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Back_Matter/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Back_Matter/18038.xml,1712,BPO
1712/Advertisement/18038,,Advertisement,1712,17120102,6403376,18038,The Spectator (1711),London,Sam. Buckley.,,...,Advertisement,1358-6149,ENG,Historical Periodicals,319,/home/ryan/lltk_data/corpora/bpo/freqs/1712/Advertisement/18038.json,/home/ryan/lltk_data/corpora/bpo/txt/1712/Advertisement/18038.txt,/home/ryan/lltk_data/corpora/bpo/xml/1712/Advertisement/18038.xml,1712,BPO


In [5]:
# import ujson

# def gettextlen(path):
#     try:
#         with open(path) as f:
#             d=ujson.load(f)
#             return sum(d.values()) if d else np.nan
#     except FileNotFoundError:
#         return np.nan
    
# text_lens = pmap(gettextlen, C.meta.path_freqs)

In [112]:
dfmeta=C.meta
dfmeta['Genre']=dfmeta.ObjectType.replace({'':'Undefined'})
dfmeta

,author,title,year,NumericPubDate,RecordID,Publication_PublicationID,Publication_Title,Publication_Qualifier,Publisher,VolumeIssue,...,LanguageCode,SourceType,num_words,path_freqs,path_txt,path_xml,_year_orig,corpus,period,Genre
id,,,,,,,,,,,,,,,,,,,,,
1010/Advertisement/2032695,,Advertisements,1010,10101,1618364638,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,ENG,Historical Periodicals,1155.0,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Advertisement/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Advertisement/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Advertisement/2032695.xml,1010,BPO,1000-1020,Advertisement
1010/Advertisement/2032690,,MAPPIN & WEBB Ltd.,1010,10101,1618367119,2032690,The Sketch,London,Illustrated London News,,...,ENG,Historical Periodicals,131.0,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Advertisement/2032690.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Advertisement/2032690.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Advertisement/2032690.xml,1010,BPO,1000-1020,Advertisement
1010/Unknown/2032695,,THE Old Etonians beat Edinburgh University by five goal four in a football match at Edinburgh on Tuesday,1010,10101,1618367761,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,ENG,Historical Periodicals,55.0,/home/ryan/lltk_data/corpora/bpo/freqs/1010/Unknown/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/Unknown/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/Unknown/2032695.xml,1010,BPO,1000-1020,Undefined
1010/News/2032690,,"GIVEN COMING-OF,-AGE FESTIVITIES, JUST LIKE A SON AND HEIR",1010,10101,1618367834,2032690,The Sketch,London,Illustrated London News,,...,ENG,Historical Periodicals,64.0,/home/ryan/lltk_data/corpora/bpo/freqs/1010/News/2032690.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/News/2032690.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/News/2032690.xml,1010,BPO,1000-1020,News
1010/News/2032695,,Graphic,1010,10101,1618372563,2032695,Illustrated Sporting and Dramatic News,London,Illustrated London News,,...,ENG,Historical Periodicals,0.0,/home/ryan/lltk_data/corpora/bpo/freqs/1010/News/2032695.json,/home/ryan/lltk_data/corpora/bpo/txt/1010/News/2032695.txt,/home/ryan/lltk_data/corpora/bpo/xml/1010/News/2032695.xml,1010,BPO,1000-1020,News
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996/Advertisement/2032697,,TURKEY,1996,19960226,1689079340,2032697,London Life,London,,,...,ENG,Historical Periodicals,233.0,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Advertisement/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Advertisement/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Advertisement/2032697.xml,1996,BPO,1980-2000,Advertisement
1996/Unknown/2032697,,BELGRAVIA DECLARES METER WAR,1996,19960226,1689080163,2032697,London Life,London,,,...,ENG,Historical Periodicals,894.0,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Unknown/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Unknown/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Unknown/2032697.xml,1996,BPO,1980-2000,Undefined
1996/Review/2032697,,The Absorbing Fire,1996,19960226,1689080953,2032697,London Life,London,,,...,ENG,Historical Periodicals,68.0,/home/ryan/lltk_data/corpora/bpo/freqs/1996/Review/2032697.json,/home/ryan/lltk_data/corpora/bpo/txt/1996/Review/2032697.txt,/home/ryan/lltk_data/corpora/bpo/xml/1996/Review/2032697.xml,1996,BPO,1980-2000,Review


In [7]:
dfmeta.year

id
1010/Advertisement/2032695    1010
1010/Advertisement/2032690    1010
1010/Unknown/2032695          1010
1010/News/2032690             1010
1010/News/2032695             1010
                              ... 
1996/Advertisement/2032697    1996
1996/Unknown/2032697          1996
1996/Review/2032697           1996
1997/News/2032690             1997
1997/Unknown/2032690          1997
Name: year, Length: 50449, dtype: int16

In [13]:
ybin=20
dfmeta=dfmeta.query('1600<year<2000')
dfmeta['period']=dfmeta.year.apply(lambda y: periodize(y,ybin))
dfmeta.period

id
1642/Unknown/2032695                1640-1660
1642/News/2032695                   1640-1660
1642/Advertisement/2032695          1640-1660
1642/General_Information/2032695    1640-1660
1681/Advertisement/18561            1680-1700
                                      ...    
1996/Advertisement/2032697          1980-2000
1996/Unknown/2032697                1980-2000
1996/Review/2032697                 1980-2000
1997/News/2032690                   1980-2000
1997/Unknown/2032690                1980-2000
Name: period, Length: 50426, dtype: object

In [30]:
dfmeta['count']=1
dfinfo1=dfmeta.groupby('period').sum().drop('year',1)
dfinfo1

,count
period,
1640-1660,4
1680-1700,174
1700-1720,493
1720-1740,758
1740-1760,1143
1760-1780,1633
1780-1800,2280
1800-1820,3773
1820-1840,5102


## Skip info

In [150]:
# get_skipgrams??

In [29]:
dfskip=get_skipgrams(calc_numlines=True)
dfskip=dfskip.query('1600<year<2000')
dfskip['period']=dfskip.year.apply(lambda y: periodize(y,ybin))
dfinfo2=dfskip.groupby('period').sum().drop('year',1)

100%|██████████| 289/289 [00:06<00:00, 45.83it/s] 


In [50]:
dfinfo = dfinfo1.join(dfinfo2).dropna()
dfinfo_str=dfinfo.applymap(lambda x: f'{int(x):,}').rename_axis('Period (20yr)')
dfinfo_str.columns=['# Texts','# Sentences','# Words']
printm(dfinfo_str.to_markdown())

| Period (20yr)   | # Texts   | # Sentences   | # Words     |
|:----------------|:----------|:--------------|:------------|
| 1680-1700       | 174       | 51,395        | 1,284,875   |
| 1700-1720       | 493       | 159,961       | 3,999,025   |
| 1720-1740       | 758       | 354,569       | 8,864,225   |
| 1740-1760       | 1,143     | 622,054       | 15,551,350  |
| 1760-1780       | 1,633     | 1,049,694     | 26,242,350  |
| 1780-1800       | 2,280     | 1,601,102     | 40,027,550  |
| 1800-1820       | 3,773     | 2,634,714     | 65,867,850  |
| 1820-1840       | 5,102     | 4,225,179     | 105,629,475 |
| 1840-1860       | 5,901     | 5,469,517     | 136,737,925 |
| 1860-1880       | 8,551     | 8,021,784     | 200,544,600 |
| 1880-1900       | 10,668    | 9,207,343     | 230,183,575 |
| 1900-1920       | 6,267     | 5,454,356     | 136,358,900 |
| 1920-1940       | 2,623     | 2,377,737     | 59,443,425  |
| 1940-1960       | 833       | 694,232       | 17,355,800  |
| 1960-1980       | 192       | 196,544       | 4,913,600   |

In [57]:
for x,y in dfinfo.sum().items(): printm(f'* {x}: {int(y):,}')

* count: 50,391

* num_lines: 42,120,181

* num_words: 1,053,004,525

In [61]:
dfmeta.nunique().sort_values()

author                           1
corpus                           1
LanguageCode                     1
VolumeIssue                      1
count                            1
SourceType                       4
ObjectType                      13
period                          17
Publication_Qualifier           25
year                           297
_year_orig                     297
Publisher                      384
ISSN                           461
Publication_PublicationID      480
Publication_Title              679
EndPage                       2372
StartPage                     2487
NumericPubDate                7230
num_words                     7834
title                        27562
RecordID                     50426
path_freqs                   50426
path_txt                     50426
path_xml                     50426
dtype: int64

In [62]:
dfmeta.title

id
1642/Unknown/2032695                            Dogs of Portugal
1642/News/2032695                          Australians at Lord's
1642/Advertisement/2032695                  WHITBREAD & CO. LTD.
1642/General_Information/2032695                           Other
1681/Advertisement/18561                           Advertisement
                                                ...             
1996/Advertisement/2032697                                TURKEY
1996/Unknown/2032697                BELGRAVIA DECLARES METER WAR
1996/Review/2032697                           The Absorbing Fire
1997/News/2032690                              A Living Wedgwood
1997/Unknown/2032690                                AFTER DINNER
Name: title, Length: 50426, dtype: object

In [64]:
print(dfinfo_str.to_markdown())

[Koselleck] (16:03:07) | Period (20yr)   | # Texts   | # Sentences   | # Words     |
|:----------------|:----------|:--------------|:------------|
| 1680-1700       | 174       | 51,395        | 1,284,875   |
| 1700-1720       | 493       | 159,961       | 3,999,025   |
| 1720-1740       | 758       | 354,569       | 8,864,225   |
| 1740-1760       | 1,143     | 622,054       | 15,551,350  |
| 1760-1780       | 1,633     | 1,049,694     | 26,242,350  |
| 1780-1800       | 2,280     | 1,601,102     | 40,027,550  |
| 1800-1820       | 3,773     | 2,634,714     | 65,867,850  |
| 1820-1840       | 5,102     | 4,225,179     | 105,629,475 |
| 1840-1860       | 5,901     | 5,469,517     | 136,737,925 |
| 1860-1880       | 8,551     | 8,021,784     | 200,544,600 |
| 1880-1900       | 10,668    | 9,207,343     | 230,183,575 |
| 1900-1920       | 6,267     | 5,454,356     | 136,358,900 |
| 1920-1940       | 2,623     | 2,377,737     | 59,443,425  |
| 1940-1960       | 833       | 694,232       |

In [65]:
dfmeta.nunique().sort_values()

author                           1
corpus                           1
LanguageCode                     1
VolumeIssue                      1
count                            1
SourceType                       4
ObjectType                      13
period                          17
Publication_Qualifier           25
year                           297
_year_orig                     297
Publisher                      384
ISSN                           461
Publication_PublicationID      480
Publication_Title              679
EndPage                       2372
StartPage                     2487
NumericPubDate                7230
num_words                     7834
title                        27562
RecordID                     50426
path_freqs                   50426
path_txt                     50426
path_xml                     50426
dtype: int64

In [139]:
genres=dfmeta.Genre.value_counts()
genres=pd.DataFrame({'Count':genres}).rename_axis('Genre')
s=genres.Count
genres['perc']=s.apply(lambda x: f'{(x/sum(s)*100):,.0f}%')
genres.Count=s.apply(lambda x: f'{x:,.0f}')
genres.columns=['# Texts','% Texts']
genres['count']=s
# print(f'\n\n{genres.to_markdown()}\n\n')
# genres

In [140]:
words=dfmeta.groupby('Genre').agg(['sum','mean']).drop('year',1)
words.columns=['_'.join(x) for x in words.columns.to_flat_index()]
words['perc']=words['num_words_sum'] / words['num_words_sum'].sum() * 100
words=words.sort_values('perc',ascending=False)
words=words.applymap(lambda x: f'{x:,.0f}')
words['perc']=words['perc'].apply(lambda x: f'{x}%')
words=words[['num_words_sum','perc','num_words_mean']]
words.columns=['# Words','% Words', '# Words (avg.)']
# print(f'\n\n{words.to_markdown()}\n\n')

In [144]:
dfinfogenre = genres.join(words).sort_values('count',ascending=False).drop('count',1)
dfinfogenre

,# Texts,% Texts,# Words,% Words,# Words (avg.)
Genre,,,,,
Undefined,"7,673",15%,"14,716,970",19%,"1,918"
General Information,"6,747",13%,"7,756,268",10%,"1,150"
Review,"6,156",12%,"19,686,239",25%,"3,198"
Back Matter,"5,166",10%,"2,682,952",3%,519
Poem,"4,968",10%,"1,972,245",3%,397
Correspondence,"4,431",9%,"6,809,535",9%,"1,537"
Front Matter,"4,126",8%,"2,379,941",3%,577
Advertisement,"3,565",7%,"3,484,090",5%,977
Fiction,"3,388",7%,"16,578,587",21%,"4,893"


In [145]:
print(dfinfogenre.to_markdown())

[Koselleck] (16:42:55) | Genre               | # Texts   | % Texts   | # Words    | % Words   | # Words (avg.)   |
|:--------------------|:----------|:----------|:-----------|:----------|:-----------------|
| Undefined           | 7,673     | 15%       | 14,716,970 | 19%       | 1,918            |
| General Information | 6,747     | 13%       | 7,756,268  | 10%       | 1,150            |
| Review              | 6,156     | 12%       | 19,686,239 | 25%       | 3,198            |
| Back Matter         | 5,166     | 10%       | 2,682,952  | 3%        | 519              |
| Poem                | 4,968     | 10%       | 1,972,245  | 3%        | 397              |
| Correspondence      | 4,431     | 9%        | 6,809,535  | 9%        | 1,537            |
| Front Matter        | 4,126     | 8%        | 2,379,941  | 3%        | 577              |
| Advertisement       | 3,565     | 7%        | 3,484,090  | 5%        | 977              |
| Fiction             | 3,388     | 7%        | 16,578,58

In [119]:

words

,"(num_words, mean)","(num_words, sum)"
Genre,,
Advertisement,977.304348,3484090.0
Back Matter,519.348045,2682952.0
Correspondence,1536.794177,6809535.0
Fiction,4893.325561,16578587.0
Front Matter,576.815560,2379941.0
General Information,1149.587669,7756268.0
News,27.491554,65100.0
Obituary,590.753846,883177.0
Poem,396.989734,1972245.0


In [95]:
dfmeta[dfmeta.ObjectType=='General Information'].index[0]

'1642/General_Information/2032695'

In [96]:
t=C.textd['1642/General_Information/2032695']
t

, _Other_ (1642) [BPO: 1642/General_Information/2032695]

In [97]:
t.txt

'####1638050891\nPostage Rates Inland 2d. Canada Id. Else where Abroad I Jd. Entered as second-class Mail matter at the Mew York Post Office         The Illustrated SPORTING ir and DRAMATIC News\n\n\n\n####1638043414\nPrinted in England for the Proprietors, the SPORTING AND DRAMATIC PUBLISHING CO., LTD., Commonwealth House, I, New Oxford Street. London. W.C.I (TcleDhone Holborn 6955).         by UUHAnb PRESS LTD., Lo.ig Acre, London, W.C.2. Agents for Australia, New Zealand, and South Africa: Gordon Gotch, Ltd. NEXT ISSUE JUNE 26.\n\n\n\n'

In [98]:
C.meta.mean()

year                         1858.426014
NumericPubDate                       inf
RecordID                             inf
Publication_PublicationID            inf
num_words                            inf
_year_orig                           inf
dtype: float64

In [103]:
C.meta.num_words=C.meta.num_words.apply(float)
C.meta.num_words.mean()

1531.6525996550972

In [108]:
C.meta.groupby('ObjectType').sum()

,year,num_words
ObjectType,,
,13944378.0,14306116.0
Advertisement,6672737.0,3484090.0
Back Matter,9596850.0,2682952.0
Correspondence,8173580.0,6809535.0
Fiction,6342945.0,16578587.0
Front Matter,7675223.0,2379941.0
General Information,12521633.0,7756268.0
News,4435574.0,65100.0
Obituary,2755545.0,883177.0
